In [1]:
import pandas as pd
import pickle
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta


from tqdm.notebook import tqdm
tqdm.pandas()
from utils import full_clean, create_subframe, create_t3_df, create_t1_df, get_parallel_edges, create_complete_df, merge_df,\
    get_parallel_edges, get_weighted_edges, get_edge_info, create_post_comment_name_id
from utils.clean_df import advanced_clean

/Users/victor/opt/anaconda3/envs/ust/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


# Import the bot list retrieved online

In [ ]:
bot_list = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/bot_list.csv')
bot_list['Username'] = bot_list.Username.apply(lambda x: x[3:])

# Import the top spammers who are considered as bots

In [ ]:
popular_author = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/10k_sub_final.csv')
popular_bot = popular_author[popular_author.is_bot == 1]

--------
<h1>Proper work</h1>

In [ ]:
# #month walker
# def month_walker(comment_df, post_df, year, month):
#     month_info = dict()
#     concat_df = merge_df(comment_df, post_df)
#     #clean dataframe
#     bot, cleaned_df = full_clean(concat_df, bot_list, popular_bot, by_parent=True)
#     #get the output df list
#     df_list = create_subframe(year, month, cleaned_df)
#     day = 1
#     for daydf in df_list:
#         #get t3_df
#         t3_name_id, t3_df = create_t3_df(daydf)
#         #get t1_df
#         t1_name_id, t1_df = create_t1_df(daydf)
#         #concat the t1 and t3 df to get the complete dataframe
#         complete_df = create_complete_df(t1_name_id, t1_df, t3_name_id, t3_df)
        
#         #get weighted edge list   
#         parallel_edges = get_parallel_edges(complete_df, daydf)
#         weighted_edges = get_weighted_edges(parallel_edges)
#         weighted_edges['inverse_weight'] = 1/weighted_edges['weight']
        
#         # get the information
#         # get the list of post author names
#         daydf = daydf[daydf['parent_id']==0]
#         post_authors = daydf['author'].to_numpy()
#         edge_info = get_edge_info(weighted_edges, post_authors)
        
        
#         #this saves the df to the month network list
#         month_info.update({day: edge_info})
#         day += 1
        
#     return month_info

In [ ]:
def month_walker(comment_df, post_df, year, month):
    # clean the comment df
    comment_bot, comment_df = full_clean(comment_df, bot_list, popular_bot, by_parent=True)
    # clean the post df
    post_bot, post_df = full_clean(post_df, bot_list, popular_bot)
    # cut the post df by month
    startdate = datetime(year, month, 1, 0)
    startdate = startdate.timestamp()
    enddate = startdate + relativedelta(month=1)
    enddate = enddate.timestamp()
    print(startdate)
    print(enddate)
    post_df = post_df[(post_df.created_utc >= startdate) & (post_df.created_utc < enddate)]
    # create the t1 and t3 id:name key value pair
    t3_name_id, t1_name_id = create_post_comment_name_id(post_df, comment_df)
    # concat the comment and post df
    concat_df = merge_df(comment_df, post_df)
    # create the t1 and t3 df
    t1_df = create_t1_df(concat_df)
    t3_df = create_t3_df(concat_df)
    # create the complete df
    complete_df = create_complete_df(t1_name_id, t1_df, t3_name_id, t3_df)
    # create the parallel and weighted edgelist
    parallel_edges = get_parallel_edges(complete_df, concat_df)
    weighted_edges = get_weighted_edges(parallel_edges)
    weighted_edges['inverse_weight'] = 1/weighted_edges['weight']
    
    return parallel_edges
    
    # save the edgelists ......

In [ ]:
fp_comment = '/Users/victor/Desktop/CS/UST_summer/01-20_comments.csv'
comment_df = pd.read_csv(fp_comment, low_memory = False)[['author', 'id', 'created_utc', 'link_id', 'parent_id']]
fp_post = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv'
post_df = pd.read_csv(fp_post, low_memory = False)[['author', 'id', 'created_utc']]

In [ ]:
parallel_edges = month_walker(comment_df, post_df, 2020, 1)

----------
<h1>Test</h1>

In [ ]:
fp_comment = '/Users/victor/Desktop/CS/UST_summer/02-20_comments.csv'
comment_df = pd.read_csv(fp_comment, low_memory = False)[['author', 'id', 'created_utc', 'link_id', 'parent_id']]
fp_post = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv'
post_df = pd.read_csv(fp_post, low_memory = False)[['author', 'id', 'created_utc']]

In [ ]:
comment_bot, comment_df = full_clean(comment_df, bot_list, popular_bot, by_parent=True)

In [ ]:
post_bot, post_df = full_clean(post_df, bot_list, popular_bot)

In [ ]:
set(comment_bot.author)

In [ ]:
set(post_bot.author)

In [ ]:
t3_name_id, t1_name_id = create_post_comment_name_id(post_df, comment_df)

In [ ]:
concat_df = merge_df(comment_df, post_df)

In [ ]:
t3_df = create_t3_df(concat_df)

In [ ]:
t1_df = create_t1_df(concat_df)

In [ ]:
complete_df = create_complete_df(t1_name_id, t1_df, t3_name_id, t3_df)

In [ ]:
author_list = complete_df.merged.iloc[0]
author_list

In [ ]:
author = author_list[0]
k,v = next((str(k), str(v)) for k, v in author.items())
type(k)
    

In [ ]:
parallel_edges = get_parallel_edges(complete_df, concat_df)
weighted_edges = get_weighted_edges(parallel_edges)
weighted_edges['inverse_weight'] = 1/weighted_edges['weight']

In [ ]:
type_table = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/type_table_final.csv')

In [ ]:
type_table

In [ ]:
type_df = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/type_df_final.csv')

In [ ]:
type_df

------
re-clean the dataframe

In [2]:
fp = '/Users/victor/Desktop/CS/UST_summer/data/monthly_edgelist/01-20-edgelist.csv'
edgelist = pd.read_csv(fp, low_memory=False)

In [3]:
edgelist

,Unnamed: 0,Source,Target,is_main,source_utc,target_utc,main_is_old
0,0,ShawnHans007,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
1,1,oAneurysMo,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
2,2,lkjhgfdsasdfghjkl,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
3,3,Momar11,Logan_Maransy,0.0,1.580427e+09,1.580425e+09,0.0
4,4,dim-mi,Logan_Maransy,0.0,1.580427e+09,1.580425e+09,0.0
...,...,...,...,...,...,...,...
883170,883170,BubblesMan36,BubblesMan36,NaN,NaN,NaN,NaN
883171,883171,Shoddy_Ad9614,Shoddy_Ad9614,NaN,NaN,NaN,NaN
883172,883172,z12332,z12332,NaN,NaN,NaN,NaN
883173,883173,yeahwhateverchris,yeahwhateverchris,NaN,NaN,NaN,NaN


In [4]:
drop_indices = set()
drop_author = set()
for i, row in edgelist.iterrows():
    source = row.Source
    target = row.Target
    
    if advanced_clean(source):
        drop_indices.add(i)
        drop_author.add(source)
    elif advanced_clean(target):
        drop_indices.add(i)
        drop_author.add(target)

In [5]:
len(drop_indices)

540

In [6]:
edgelist.drop(index=list(drop_indices), inplace=True)

In [7]:
edgelist

,Unnamed: 0,Source,Target,is_main,source_utc,target_utc,main_is_old
0,0,ShawnHans007,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
1,1,oAneurysMo,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
2,2,lkjhgfdsasdfghjkl,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
3,3,Momar11,Logan_Maransy,0.0,1.580427e+09,1.580425e+09,0.0
4,4,dim-mi,Logan_Maransy,0.0,1.580427e+09,1.580425e+09,0.0
...,...,...,...,...,...,...,...
883170,883170,BubblesMan36,BubblesMan36,NaN,NaN,NaN,NaN
883171,883171,Shoddy_Ad9614,Shoddy_Ad9614,NaN,NaN,NaN,NaN
883172,883172,z12332,z12332,NaN,NaN,NaN,NaN
883173,883173,yeahwhateverchris,yeahwhateverchris,NaN,NaN,NaN,NaN


----------

In [11]:
path = '/Users/victor/Desktop/CS/UST_summer/data/monthly_edgelist_2021'
for file in os.listdir(path):
    if file.endswith('.csv'):
        edgelist = pd.read_csv(os.path.join(path, file), low_memory=False)[['Source', 'Target', 'is_main', 'source_utc', 'target_utc','main_is_old']]
        # clean starts here
        # drop_indices = set()
        # for i, row in edgelist.iterrows():
        #     source = row.Source
        #     target = row.Target
        #     if advanced_clean(source) or advanced_clean(target):
        #         drop_indices.add(i)
                
        # edgelist.drop(index=list(drop_indices), inplace=True)
        
        edgelist.to_csv(os.path.join(path, file))

In [12]:
edge0120 = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/data/monthly_edgelist_2020/01-20-edgelist.csv')

In [13]:
edge0120

,Unnamed: 0,Source,Target,is_main,source_utc,target_utc,main_is_old
0,0,ShawnHans007,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
1,1,oAneurysMo,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
2,2,lkjhgfdsasdfghjkl,Logan_Maransy,0.0,1.580426e+09,1.580425e+09,0.0
3,3,Momar11,Logan_Maransy,0.0,1.580427e+09,1.580425e+09,0.0
4,4,dim-mi,Logan_Maransy,0.0,1.580427e+09,1.580425e+09,0.0
...,...,...,...,...,...,...,...
882630,882630,BubblesMan36,BubblesMan36,NaN,NaN,NaN,NaN
882631,882631,Shoddy_Ad9614,Shoddy_Ad9614,NaN,NaN,NaN,NaN
882632,882632,z12332,z12332,NaN,NaN,NaN,NaN
882633,882633,yeahwhateverchris,yeahwhateverchris,NaN,NaN,NaN,NaN
